# Multi-Layer Perceptron

In [2]:
from sklearn.metrics import r2_score, mean_absolute_error
from dataset_storing import k_fold, fire_X, fire_centroid_x, fire_centroid_y, fire_area
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


this is working


2024-01-01 13:57:48.160851: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-01 13:57:48.195993: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-01 13:57:48.453993: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 13:57:48.454069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 13:57:48.512087: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Predicting area

In [3]:
fire_X.head(1)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,17.88,15.862,15.863,15.862,16.433,17.88,15.862,17.88,17.88,16.433,...,20.896,18.781,20.896,18.781,18.781,20.896,20.896,18.781,21.254,20.896


In [6]:
split_indices = k_fold.split(fire_X, fire_centroid_x)

imputer = SimpleImputer()
fire_X = imputer.fit_transform(fire_X)

accuracy_area_dict = {
    "mean_error_train": [],
    "mean_error_test": [],
    "r2_score_train": [],
    "r2_score_test": []
}

for i, (train, test) in enumerate(split_indices):
    model = MLPRegressor(hidden_layer_sizes=(732, 366, 183, 91), learning_rate='adaptive', max_iter=10)
    
    train_X = fire_X[train]
    train_y = fire_area.iloc[train]
    test_X = fire_X[test]
    test_y = fire_area.iloc[test]

    model.fit(train_X, train_y)
    
    print(f"Trial {i}:")
    y_pred = model.predict(test_X)
    y_pred_train = model.predict(train_X)
    print(r2_score(test_y, y_pred))
    
    accuracy_area_dict["mean_error_train"].append(np.float128(mean_absolute_error(train_y, y_pred_train)))
    accuracy_area_dict["mean_error_test"].append(np.float128(mean_absolute_error(test_y, y_pred)))
    accuracy_area_dict["r2_score_train"].append(np.float128(r2_score(train_y, y_pred_train)))
    accuracy_area_dict["r2_score_test"].append(np.float128(r2_score(test_y, y_pred)))


/home/hummingbee57/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Trial 0:
0.00029616359535944436


/home/hummingbee57/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Trial 1:
0.0040010081449275425


/home/hummingbee57/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Trial 2:
0.0007953186937773582


/home/hummingbee57/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Trial 3:
0.0016245818661970723


/home/hummingbee57/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Trial 4:
8.408029941409101e-05


In [ ]:
accuracy_area_df = pd.DataFrame(accuracy_area_dict, index=range(1, 26))
accuracy_area_df.to_csv("data/mlp_area.csv")

X coord of the centroid

In [ ]:
split_indices = k_fold.split(fire_X, fire_centroid_x)

accuracy_x_dict = {
    "mean_error_train": [],
    "mean_error_test": [],
    "r2_score_train": [],
    "r2_score_test": []
}

for i, (train, test) in enumerate(split_indices):
    model = Sequential([ 
        Dense(256, activation='relu'),  
        Dense(128, activation='relu'),
        Dense(64, activation='sigmoid'),
        Dense(1, activation='linear'),   
    ]) 
    
    model.compile(loss='mse', optimizer='adam')
    
    train_X = fire_X.iloc[train].values.tolist()
    train_y = fire_centroid_x.iloc[train].values.tolist()
    test_X = fire_X.iloc[test].values.tolist()
    test_y = fire_centroid_x.iloc[test].values.reshape((len(test), 1))

    model.fit(train_X, train_y, epochs=10)
    
    print(f"Trial {i}:")
    y_pred = model.predict(test_X)
    y_pred_train = model.predict(train_X)
    print(r2_score(test_y, y_pred))
    
    accuracy_x_dict["mean_error_train"].append(np.float128(mean_absolute_error(train_y, y_pred_train)))
    accuracy_x_dict["mean_error_test"].append(np.float128(mean_absolute_error(test_y, y_pred)))
    accuracy_x_dict["r2_score_train"].append(np.float128(r2_score(train_y, y_pred_train)))
    accuracy_x_dict["r2_score_test"].append(np.float128(r2_score(test_y, y_pred)))


Epoch 1/10
331/331 [==============================] - 2s 4ms/step - loss: 36368191488.0000
Epoch 2/10
331/331 [==============================] - 1s 4ms/step - loss: 36368166912.0000
Epoch 3/10
331/331 [==============================] - 1s 4ms/step - loss: 36368171008.0000
Epoch 4/10
331/331 [==============================] - 1s 4ms/step - loss: 36368158720.0000
Epoch 5/10
331/331 [==============================] - 1s 4ms/step - loss: 36368158720.0000
Epoch 6/10
331/331 [==============================] - 1s 4ms/step - loss: 36368125952.0000
Epoch 7/10
331/331 [==============================] - 1s 4ms/step - loss: 36368150528.0000
Epoch 8/10
331/331 [==============================] - 1s 4ms/step - loss: 36368130048.0000
Epoch 9/10
331/331 [==============================] - 1s 4ms/step - loss: 36368121856.0000
Epoch 10/10
331/331 [==============================] - 1s 4ms/step - loss: 36368105472.0000
Trial 0:
331/331 [==============================] - 0s 1ms/step
-0.0005235538188579358
Ep

In [ ]:
accuracy_x_df = pd.DataFrame(accuracy_x_dict, index=range(1, 26))
accuracy_x_df.to_csv("data/mlp_cent_x.csv")

y coord of the centroid

In [ ]:
split_indices = k_fold.split(fire_X, fire_centroid_x)

accuracy_y_dict = {
    "mean_error_train": [],
    "mean_error_test": [],
    "r2_score_train": [],
    "r2_score_test": []
}

for i, (train, test) in enumerate(split_indices):
    model = Sequential([ 
        Dense(256, activation='relu'),  
        Dense(128, activation='relu'),
        Dense(64, activation='sigmoid'),
        Dense(1, activation='linear'),   
    ]) 
    
    model.compile(loss='mse', optimizer='adam')
    
    train_X = fire_X.iloc[train].values.tolist()
    train_y = fire_centroid_x.iloc[train].values.tolist()
    test_X = fire_X.iloc[test].values.tolist()
    test_y = fire_centroid_x.iloc[test].values.reshape((len(test), 1))

    model.fit(train_X, train_y, epochs=10)
    
    print(f"Trial {i}:")
    y_pred = model.predict(test_X)
    y_pred_train = model.predict(train_X)
    print(r2_score(test_y, y_pred))
    
    accuracy_y_dict["mean_error_train"].append(np.float128(mean_absolute_error(train_y, y_pred_train)))
    accuracy_y_dict["mean_error_test"].append(np.float128(mean_absolute_error(test_y, y_pred)))
    accuracy_y_dict["r2_score_train"].append(np.float128(r2_score(train_y, y_pred_train)))
    accuracy_y_dict["r2_score_test"].append(np.float128(r2_score(test_y, y_pred)))


Epoch 1/10
331/331 [==============================] - 2s 4ms/step - loss: 36368207872.0000
Epoch 2/10
331/331 [==============================] - 2s 5ms/step - loss: 36368199680.0000
Epoch 3/10
331/331 [==============================] - 1s 4ms/step - loss: 36368158720.0000
Epoch 4/10
331/331 [==============================] - 1s 5ms/step - loss: 36368154624.0000
Epoch 5/10
331/331 [==============================] - 2s 5ms/step - loss: 36368150528.0000
Epoch 6/10
331/331 [==============================] - 2s 5ms/step - loss: 36368154624.0000
Epoch 7/10
331/331 [==============================] - 1s 4ms/step - loss: 36368138240.0000
Epoch 8/10
331/331 [==============================] - 1s 4ms/step - loss: 36368105472.0000
Epoch 9/10
331/331 [==============================] - 1s 4ms/step - loss: 36368113664.0000
Epoch 10/10
331/331 [==============================] - 1s 4ms/step - loss: 36368113664.0000
Trial 0:
331/331 [==============================] - 1s 2ms/step
-0.0005239187109336019
Ep

In [ ]:
accuracy_y_df = pd.DataFrame(accuracy_y_dict, index=range(1, 26))
accuracy_y_df.to_csv("data/mlp_cent_y.csv")